<h1>Casestudy Eric Bühler - Aachen </h1>

In [2]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import collections
from datetime import datetime, timedelta
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import numpy as np
import pandas as pd
import configparser
import openai


In [13]:

def scrape_wgs(pages=2):
    anzeigen=[]
    for i in range(pages):
        url = f"https://www.wg-gesucht.de/wg-zimmer-in-Aachen.1.0.1.{i}.html?offer_filter=1&city_id=1&sort_order=0&noDeact=1&categories%5B%5D=0&pagination=1&pu="
        # Sends a GET request to the URL to retrieve its content
        response = requests.get(url)
        assert response.status_code == 200
        #print(response.text)
        # Parse results
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting all Anzeigen
        anzeigen_raw = soup.find_all('div', class_='col-sm-8 card_body')
        # Loop through each anzeige section and build dictionaries of them
        raw_text = ''
        for anzeige in anzeigen_raw:
            titel = anzeige.find('a').get_text().replace('\n', '').strip()
            preis = anzeige.find('div', class_='col-xs-3').get_text().replace('€','').strip()
            frei_ab = anzeige.find('div', class_='col-xs-5 text-center').get_text().replace('\n', '').split('-')[0].strip()
            qm = anzeige.find('div', class_='col-xs-3 text-right').get_text().replace('\n', '').replace('m²','').strip()
            wg_groesse = anzeige.find('div', class_='col-xs-11').get_text().split('er')[0].split()[0]
            stadtteil = anzeige.find('div', class_='col-xs-11').get_text().strip().split('|')[1].replace('\n', '').strip().split()
            #uncomment if i get rich
            #plz = open_AI_call()
            
            online_seit = anzeige.find('div', class_='col-xs-9').get_text().split('Online:')[1].strip()
            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit="00:00"

            if "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"00:{minutes.zfill(2)}"  # Ensuring minutes are two digits

            if "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{hours.zfill(2)}:00"  # Ensuring hours are two digits

            if "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                hours = int(days)*24
                online_seit = f"{hours}:00"
            
            anzeigen.append({'titel': titel,'preis' : preis,'frei_ab':frei_ab, 'qm': qm,'wg_groesse':wg_groesse,'stadtteil':stadtteil,'online_seit':online_seit})
    return pd.DataFrame(anzeigen)

# Display the DataFrame
#test_data= scrape_wgs(10)
display(test_data)

,titel,preis,frei_ab,qm,wg_groesse,stadtteil,online_seit
0,Studentenzimmer in direkter Uninähe,265,01.09.2024,20,11,"[Aachen, Aachen]",00:08
1,Zimmer in bester Studentenverbindung,260,12.06.2024,15,17,"[Aachen, Aachen]",00:09
2,Zimmer in 2-Personen WG,417,09.06.2024,20,2,"[Aachen, Aachen]",00:10
3,2 schöne möblierte Zimmer in Aachen-Mitte mit ...,279,09.05.2024,16,14,"[Aachen, Aachen]",00:13
4,WG-Zimmer in bester Lage für Studenten,200,01.07.2024,14,6,"[Aachen, Aachen]",00:13
...,...,...,...,...,...,...,...
175,16m^2 Zimmer in top Lage,325,15.08.2024,16,4,"[Aachen, Aachen]",96:00
176,WG-Zimmer in zentraler Lage,470,01.07.2024,15,3,"[Aachen, Mitte]",96:00
177,Möbliertes WG Zimmer in ansprechender Doppelha...,500,07.06.2024,14,4,"[Aachen, Steppenberg]",96:00
178,Zimmer in bester Lage in der vierer WG mit den...,400,07.06.2024,12,4,"[Aachen, Aachen-Mitte]",96:00


In [ ]:
def get_anzeigen_from_html(html_list):
    anzeigen = []
    for soup in html_list['html']:
        try:
            titel_element = soup.find('h1', class_='headline headline-detailed-view-title')
            titel = titel_element.get_text().replace('\n', '').strip() if titel_element else 'Unknown'

            wg_groesse_element = soup.find('span', class_='mr5')
            wg_groesse = wg_groesse_element['title'].split('er')[0].split()[0] if (wg_groesse_element and 'title' in wg_groesse_element.attrs) else 'Unknown'

            key_facts = soup.find_all('b', class_='key_fact_value')
            groesse = key_facts[0].get_text().split('m')[0].strip() if key_facts and len(key_facts) > 0 else 'Unknown'
            miete = key_facts[1].get_text().split('€')[0].strip() if key_facts and len(key_facts) > 1 else 'Unknown'

            minor_facts = soup.find_all('span', class_='section_panel_value')
            kaltmiete = minor_facts[0].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 0 else 'Unknown'
            nebenkosten = minor_facts[1].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 1 else 'Unknown'
            sonstige_kosten = minor_facts[2].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 2 else 'Unknown'
            kaution = minor_facts[3].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 3 else 'Unknown'
            abloesevereinb = minor_facts[4].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 4 else 'Unknown'
            frei_ab = minor_facts[5].get_text().strip() if minor_facts and len(minor_facts) > 5 else 'Unknown'

            adresse = soup.find_all('a', href='#mapContainer')
            strasse = adresse[0].get_text().strip().split()[0] if adresse else 'Unknown'
            plz = next((text for text in adresse[0].get_text().strip().split() if re.fullmatch(r'\d{5}', text)), 'Unknown') if adresse else 'Unknown'

            online_seit = soup.find('b', style='color: #218700;').get_text().strip() if soup.find('b', style='color: #218700;') else 'Unknown'
            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit = "00:00"
            elif "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"00:{minutes.zfill(2)}"
            elif "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{hours.zfill(2)}:00"
            elif "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                hours = int(days)*24
                online_seit = f"{hours}:00"

            zimmer = soup.find('div', id='freitext_0')
            zimmer = zimmer.find('p').get_text().strip() if zimmer else 'Unknown'

            lage = soup.find('div', id='freitext_1')
            lage = lage.find('p').get_text().strip() if lage else 'Unknown'

            wg_leben = soup.find('div', id='freitext_2')
            wg_leben = wg_leben.find('p').get_text().strip() if wg_leben else 'Unknown'

            sonstiges = soup.find('div', id='freitext_3')
            sonstiges = sonstiges.find('p').get_text().strip() if sonstiges else 'Unknown'

            link = html_list['link'].iloc[0]

            anzeigen.append({
                'titel': titel,
                'bewohner': wg_groesse,
                'groesse': groesse,
                'miete': miete,
                'strasse': strasse,
                'plz': plz,
                'online_seit': online_seit,
                'zimmer': zimmer,
                'lage': lage,
                'wg_leben': wg_leben,
                'sonstiges': sonstiges,
                'link': link
            })

        except Exception as e:
            print(f"An error occurred: {e}, link: {html_list['link'].iloc[0]}")
            continue

    return pd.DataFrame(anzeigen)

links = get_wg_links(1)
anzeigen_html = get_anzeigen_html(links)
anzeigen = get_anzeigen_from_html(anzeigen_html)
display(anzeigen)

In [19]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import collections
from datetime import datetime, timedelta
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import numpy as np
import pandas as pd
import configparser
import openai


SCRAPER_API_KEY = '33141f1d2afa5cf3c5dfe67546f6aa8b'  # Replace with your Scraper API key

def get_wg_links(pages=2):
    links = []
    for i in range(pages):
        url = f"https://www.wg-gesucht.de/wg-zimmer-in-Aachen.1.0.1.{i}.html?offer_filter=1&city_id=1&sort_order=0&noDeact=1&categories%5B%5D=0&pagination=1&pu="
        scraper_api_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={url}"
        response = requests.get(scraper_api_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve {url}, status code: {response.status_code}")
            print(response.text)
            continue

        # Parse results
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting all Anzeigen
        anzeigen_raw = soup.find_all('div', class_='col-sm-8 card_body')
        for anzeige in anzeigen_raw:
            anchor_tag = anzeige.find('a')
            if anchor_tag and 'href' in anchor_tag.attrs:
                links.append("https://www.wg-gesucht.de" + anchor_tag['href'])

    return links

def get_anzeigen_html(links):
    anzeigen_html = []
    for link in links:
        scraper_api_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={link}"
        response = requests.get(scraper_api_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve {link}, status code: {response.status_code}")
            print(response.text)
            continue

        # Store the BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')
        anzeigen_html.append({'html':soup,'link':link})
    return pd.DataFrame(anzeigen_html)

def get_anzeigen_from_html(html_list):
    anzeigen = []
    for soup in html_list['html']:
        try:
            titel_element = soup.find('h1', class_='headline headline-detailed-view-title')
            titel = titel_element.get_text().replace('\n', '').strip() if titel_element else 'Unknown'

            wg_groesse_element = soup.find('span', class_='mr5')
            wg_groesse = wg_groesse_element['title'].split('er')[0].split()[0] if (wg_groesse_element and 'title' in wg_groesse_element.attrs) else 'Unknown'

            key_facts = soup.find_all('b', class_='key_fact_value')
            groesse = key_facts[0].get_text().split('m')[0].strip() if key_facts and len(key_facts) > 0 else 'Unknown'
            miete = key_facts[1].get_text().split('€')[0].strip() if key_facts and len(key_facts) > 1 else 'Unknown'

            minor_facts = soup.find_all('span', class_='section_panel_value')
            kaltmiete = minor_facts[0].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 0 else 'Unknown'
            nebenkosten = minor_facts[1].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 1 else 'Unknown'
            sonstige_kosten = minor_facts[2].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 2 else 'Unknown'
            kaution = minor_facts[3].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 3 else 'Unknown'
            abloesevereinb = minor_facts[4].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 4 else 'Unknown'
            frei_ab = minor_facts[5].get_text().strip() if minor_facts and len(minor_facts) > 5 else 'Unknown'

            adresse = soup.find_all('a', href='#mapContainer')
            strasse = adresse[0].get_text().strip().split()[0] if adresse else 'Unknown'
            plz = next((text for text in adresse[0].get_text().strip().split() if re.fullmatch(r'\d{5}', text)), 'Unknown') if adresse else 'Unknown'

            online_seit = soup.find('b', style='color: #218700;').get_text().strip() if soup.find('b', style='color: #218700;') else 'Unknown'
            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit = "00:00"
            elif "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"00:{minutes.zfill(2)}"
            elif "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{hours.zfill(2)}:00"
            elif "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                hours = int(days)*24
                online_seit = f"{hours}:00"

            zimmer = soup.find('div', id='freitext_0')
            zimmer = zimmer.find('p').get_text().strip() if zimmer else 'Unknown'

            lage = soup.find('div', id='freitext_1')
            lage = lage.find('p').get_text().strip() if lage else 'Unknown'

            wg_leben = soup.find('div', id='freitext_2')
            wg_leben = wg_leben.find('p').get_text().strip() if wg_leben else 'Unknown'

            sonstiges = soup.find('div', id='freitext_3')
            sonstiges = sonstiges.find('p').get_text().strip() if sonstiges else 'Unknown'

            link = html_list['link'].iloc[0]

            anzeigen.append({
                'titel': titel,
                'bewohner': wg_groesse,
                'groesse': groesse,
                'miete': miete,
                'strasse': strasse,
                'plz': plz,
                'online_seit': online_seit,
                'zimmer': zimmer,
                'lage': lage,
                'wg_leben': wg_leben,
                'sonstiges': sonstiges,
                'link': link
            })

        except Exception as e:
            print(f"An error occurred: {e}, link: {html_list['link'].iloc[0]}")
            continue
            # Specify the file path
    file_path = 'anzeigen.csv'

    # Save DataFrame to CSV
    pd.DataFrame(anzeigen).to_csv(file_path, index=False, encoding='utf-8')

    return pd.DataFrame(anzeigen)


#links = get_wg_links()
#anzeigen_html = get_anzeigen_html(links)
anzeigen = get_anzeigen_from_html(anzeigen_html)
display(anzeigen)


,titel,bewohner,groesse,miete,strasse,plz,online_seit,zimmer,lage,wg_leben,sonstiges,link
0,Studentenwohnheim mit freundschaftlicher Gemei...,11,20,215,Kruppstraße,52072,00:07,Hallo zusammen! \n\r\n Wir sind eine engagiert...,Unknown,Unknown,Unknown,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
1,WG-Zimmer in bester Lage für Studenten,6,14,200,Hexenberg,52074,00:09,(Nach Absprache auch früherer oder späterer Ei...,Das Haus liegt in bester Aachener Lage von Ver...,"Wir sind eine katholische, nicht-schlagende, p...",Wenn du Interesse an einem Zimmer auf unserem ...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
2,2 schöne möblierte Zimmer in Aachen-Mitte mit ...,14,16,279,Zollernstraße,52070,00:12,Das Haus liegt im schönen Frankenberger Vierte...,Das Haus liegt in der Zollernstraße 25 im Fran...,Die Mehrheit der Hausbewohner sind Mitglied im...,Bist Du neugierig geworden? \r\n Schreib uns ...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
3,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
4,Möbliertes Zimmer in Uninähe,11,21,265,Salvatorstr.,52070,00:13,Du studierst an der RWTH-Aachen und vorzugswei...,Das Haus befindet sich in fußläufiger Nähe zum...,Wir sind die Aachener Burschenschaft Teutonia ...,"Da wir täglich viele Anfragen bekommen, freut ...",https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
...,...,...,...,...,...,...,...,...,...,...,...,...
395,WG ZIMMER / Room Women shared Flat,5,12,460,Reichsweg,52068,Unknown,"460€ All in Miete \n\r\n ✅Möbel (Tisch, Schran...",Unknown,Unknown,Unknown,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
396,Room in Aachen for 2 month,3,50,500,Lintertstr,52078,Unknown,"it's available for two month, there is a possi...",Unknown,Two young men from Germany and India,Unknown,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
397,Zimmer in 3er-WG mit perfekter Lage bei Aachen...,3,16,441,Im,52064,Unknown,Das Zimmer ist ca. 16 m² groß und rechteckig g...,Die Lage ist ideal für alle Studierenden. In 2...,"Deine zukünftigen Mitbewohner sind Tobias (24,...",Wir sind keine Zweck WG und suchen daher einen...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
398,WG Zimmer Aachen,2,12,430,Südstraße,52064,Unknown,Die 3-Zimmer Wohnung hat ein großes Wohnzimmer...,Von der Wohnung gibt es Anbindungen in alle Ri...,Unknown,Unknown,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...


In [21]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


anzeigen=np.asarray(anzeigen)
# Assuming 'anzeigen' DataFrame is already prepared with relevant data

# Convert 'plz' to categorical and create dummy variables
anzeigen['plz'] = anzeigen['plz'].astype('category')
plz_dummies = pd.get_dummies(anzeigen['plz'], drop_first=True, prefix='plz')

# Concatenate dummy variables with original DataFrame
anzeigen = pd.concat([anzeigen, plz_dummies], axis=1)

# Prepare X (independent variables) and y (dependent variable)
X = anzeigen[['bewohner', 'groesse'] + list(plz_dummies.columns)]
X = X.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric

y = pd.to_numeric(anzeigen['miete'], errors='coerce')  # Ensure 'miete' is numeric

# Clean NaN values if present
X = X.dropna()
y = y[X.index]  # Ensure y matches the cleaned X indices

# Add intercept to X
X = sm.add_constant(X)

# Fit OLS regression model
model = sm.OLS(y, X)
results = model.fit()

# Print summary of regression results
print(results.summary())


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices